In [1]:
import pandas as pd
from os import path
import joblib

In [2]:
# load data
DATA_DIR = '/Users/trdny_josephjungermann/Documents/the_real_deal/20250515_DFW_ML_HousingPrices/'
X = pd.read_csv(path.join(DATA_DIR, 'DFW_ALL_Data.csv'), low_memory=False)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88974 entries, 0 to 88973
Data columns (total 30 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   mlsId                88920 non-null  float64
 1   price_amount         88963 non-null  float64
 2   formattedStreetLine  88974 non-null  object 
 3   city                 88970 non-null  object 
 4   state                88974 non-null  object 
 5   county               88974 non-null  object 
 6   zip                  88974 non-null  int64  
 7   latitude             88974 non-null  float64
 8   longitude            88974 non-null  float64
 9   location             88562 non-null  object 
 10  list_agent_name_1    88128 non-null  object 
 11  list_broker_name_1   88117 non-null  object 
 12  list_agent_name_2    11893 non-null  object 
 13  list_broker_name_2   11891 non-null  object 
 14  buy_agent_name_1     88113 non-null  object 
 15  buy_broker_name_1    88104 non-null 

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
# drop the following columns: latitude, longitude, hoaDues_amount, listingAddedDate, lastSoldDate, addressInfo_full, brokers, propertyId, listingId
X.drop(['latitude', 'longitude', 'hoaDues_amount', 'listingAddedDate', 'lastSoldDate', 'addressInfo_full', 'brokers', 'propertyId', 'listingId', 'state', 'zip', 'mlsId'], axis=1, inplace=True)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88974 entries, 0 to 88973
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   price_amount         88963 non-null  float64
 1   formattedStreetLine  88974 non-null  object 
 2   city                 88970 non-null  object 
 3   county               88974 non-null  object 
 4   location             88562 non-null  object 
 5   list_agent_name_1    88128 non-null  object 
 6   list_broker_name_1   88117 non-null  object 
 7   list_agent_name_2    11893 non-null  object 
 8   list_broker_name_2   11891 non-null  object 
 9   buy_agent_name_1     88113 non-null  object 
 10  buy_broker_name_1    88104 non-null  object 
 11  buy_agent_name_2     1360 non-null   object 
 12  buy_broker_name_2    1355 non-null   object 
 13  url                  88974 non-null  object 
 14  beds                 88952 non-null  float64
 15  baths                88915 non-null 

In [5]:
# drop any rows from dataset that has a missing sale price
# save target column to y and drop from X
X.dropna(axis=0, subset=['price_amount'], inplace=True)
y = X.price_amount
X.drop(['price_amount'], axis=1, inplace=True)

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88963 entries, 0 to 88973
Data columns (total 17 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   formattedStreetLine  88963 non-null  object 
 1   city                 88960 non-null  object 
 2   county               88963 non-null  object 
 3   location             88552 non-null  object 
 4   list_agent_name_1    88117 non-null  object 
 5   list_broker_name_1   88106 non-null  object 
 6   list_agent_name_2    11892 non-null  object 
 7   list_broker_name_2   11890 non-null  object 
 8   buy_agent_name_1     88103 non-null  object 
 9   buy_broker_name_1    88093 non-null  object 
 10  buy_agent_name_2     1360 non-null   object 
 11  buy_broker_name_2    1355 non-null   object 
 12  url                  88963 non-null  object 
 13  beds                 88941 non-null  float64
 14  baths                88904 non-null  float64
 15  sqft_amount          88945 non-null  floa

In [7]:
# break off data into train_test_split
X_train_full, X_valid_full, y_train, y_valid = train_test_split(X, y, train_size = 0.8, test_size= 0.2,
                                                                random_state=0)

In [8]:
# select low cardinality object columns. Set value to 15 to include 13 different counties in our training data
low_card_columns = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 15 and
                    X_train_full[cname].dtype == 'object']
low_card_columns

['county']

In [9]:
# select all the numerical columns. In this case: zip, beds, bath, sqft_amount, yearBuilt
num_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]
num_cols

['beds', 'baths', 'sqft_amount', 'yearBuilt']

In [10]:
# Keep selected columns only
my_cols = low_card_columns + num_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [11]:
# One-hot encode the data (to shorten the code, we use pandas)
X_train = pd.get_dummies(X_train)
X_valid = pd.get_dummies(X_valid)
X_train, X_valid = X_train.align(X_valid, join='left', axis=1)

In [12]:
X_train

,beds,baths,sqft_amount,yearBuilt,county_Collin,county_Dallas,county_Denton,county_Ellis,county_Hood,county_Hunt,county_Johnson,county_Kaufman,county_Parker,county_Rockwall,county_Somervell,county_Tarrant,county_Wise
20993,3.0,2.0,1561.0,1999.0,False,False,True,False,False,False,False,False,False,False,False,False,False
75503,3.0,2.0,1604.0,2024.0,True,False,False,False,False,False,False,False,False,False,False,False,False
27939,4.0,3.5,2171.0,2024.0,False,False,True,False,False,False,False,False,False,False,False,False,False
5726,3.0,2.0,1612.0,1950.0,False,True,False,False,False,False,False,False,False,False,False,False,False
26294,4.0,3.0,2423.0,2018.0,False,False,True,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21245,4.0,2.0,1818.0,2021.0,False,False,True,False,False,False,False,False,False,False,False,False,False
45896,3.0,2.0,1605.0,2024.0,False,False,False,False,False,False,False,False,False,True,False,False,False
42617,3.0,2.0,1389.0,2004.0,False,False,False,False,False,False,False,False,True,False,False,False,False
43571,3.0,2.5,2152.0,2024.0,False,False,False,False,False,False,False,False,True,False,False,False,False


In [17]:
# Begin building model with XGboost
# Make a simple model to start called my_model_1. We'll improve as we go with my_model_2, 3, 4,etc.
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

# # Model 1
# my_model_1 = XGBRegressor(random_state=0)
# my_model_1.fit(X_train, y_train)

# # make prediction
# predictions_1 = my_model_1.predict(X_valid)

# # print mean_absolute_error
# mae_1 = mean_absolute_error(predictions_1, y_valid)
# print('Mean Absolute Error:', mae_1)

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/Users/trdny_josephjungermann/Documents/the_real_deal/20250515_DFW_ML_HousingPrices/.venv/lib/python3.10/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <948FC7F9-7446-3923-BB9F-85890E78C765> /Users/trdny_josephjungermann/Documents/the_real_deal/20250515_DFW_ML_HousingPrices/.venv/lib/python3.10/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt

In [14]:
# # Model 2
# # fit a new model adding estimators, early stop rounds and learning rate
# my_model_2 = XGBRegressor(n_estimators=100, learning_rate=0.05, early_stopping_rounds=5, random_state=0)

# my_model_2.fit(X_train, y_train,
#                eval_set=[(X_valid,y_valid)],
#                verbose=False)

# #make prediction
# predictions_2 = my_model_2.predict(X_valid)

# mae_2 = mean_absolute_error(predictions_2, y_valid)
# print('Mean Absolute Error 2:', mae_2)

In [19]:
brew install libomp

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2760538136.py, line 1)

In [15]:
# fit a new model adding estimators, early stop rounds and learning rate
my_model_3 = XGBRegressor(n_estimators=200, learning_rate=0.05, early_stopping_rounds=20, random_state=0)

my_model_3.fit(X_train, y_train,
               eval_set=[(X_valid,y_valid)],
               verbose=False)

#make prediction
predictions_3 = my_model_3.predict(X_valid)

mae_3 = mean_absolute_error(predictions_3, y_valid)
print('Mean Absolute Error 3:', mae_3)

NameError: name 'XGBRegressor' is not defined

In [ ]:
# save model to pkl file so that we can upload it to dash interactive slider
# Save model
joblib.dump(my_model_3, "/Users/trdny_josephjungermann/Documents/the_real_deal/20250515_DFW_ML_HousingPrices/DFW_house_price_model.pkl")
print("Model saved to house_price_model.pkl")

Model saved to house_price_model.pkl


## Ways to improve the model:
### 1. Fill in empty values for features: 'beds', 'baths', 'sqft_amount', 'yearBuilt'